In [1]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv(r"C:\Users\user\Diabetes-Prediction\expirements\Diabetes_prediction.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Diagnosis
0,2,115.863387,56.410731,24.336736,94.385783,26.455940,0.272682,20.100494,0
1,2,92.490122,70.615520,23.443591,138.652426,23.910167,0.665160,44.912281,0
2,1,88.141469,63.262618,23.404364,149.358082,21.948250,0.676022,48.247873,1
3,2,108.453101,67.793632,20.751580,108.751638,24.209304,0.289636,42.749868,0
4,1,127.849443,94.725685,22.603078,25.269987,32.997477,0.601315,32.797789,0


In [7]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Diagnosis                   0
dtype: int64

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               1000 non-null   int64  
 1   Glucose                   1000 non-null   float64
 2   BloodPressure             1000 non-null   float64
 3   SkinThickness             1000 non-null   float64
 4   Insulin                   1000 non-null   float64
 5   BMI                       1000 non-null   float64
 6   DiabetesPedigreeFunction  1000 non-null   float64
 7   Age                       1000 non-null   float64
 8   Diagnosis                 1000 non-null   int64  
dtypes: float64(7), int64(2)
memory usage: 70.4 KB


In [11]:
import hopsworks

project = hopsworks.login()

Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/516244


In [12]:
api_key = 'Te1uRq1N86UX5tGZ.oSTwrDFOLuUmoybnMHQGVXKMTjoZOzu7VfpIYYvGQWRYEOvMkIvLUFCIKJtpcdf9'
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [13]:
fs.name

'ajinravi_featurestore'

In [6]:
import sys
sys.path.append(r'C:\Users\user\Diabetes-Prediction\src\components')

import os
from dotenv import load_dotenv
from components.hopsworks_connect import FeatureStoreManager
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from joblib import dump

load_dotenv()

hopsworks_api_key_value: str = os.environ.get("HOPSWORKS_API_KEY_VALUE")
print("1")

manager = FeatureStoreManager(hopsworks_api_key_value)
print("2")

users_fg = manager.get_feature_group("diabetecs_data", version=1)
print("3")

query = users_fg.select_all()
print("4")

df = query.read()
print("done")

1
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/516244
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.
2
3
4


Finished: Reading data from Hopsworks, using Hive (3.73s) 
done


In [7]:
df

,pregnancies,glucose,bloodpressure,skinthickness,insulin,bmi,diabetespedigreefunction,age,diagnosis
0,1,132.678964,85.951345,22.719215,47.476594,22.973573,0.189495,20.753330,0
1,1,60.714150,64.082842,24.689064,111.562745,19.464280,0.250560,44.421874,1
2,0,115.983576,63.524499,22.855729,61.212388,28.262363,0.647677,18.702753,1
3,1,93.749145,82.365700,22.518677,93.472520,19.982207,0.300327,52.230774,0
4,0,86.252314,55.802301,22.825742,-0.205205,21.513624,0.154792,56.561546,0
...,...,...,...,...,...,...,...,...,...
995,0,113.633814,72.228418,22.326829,69.771570,28.171251,0.646513,44.811147,1
996,5,111.997186,76.741777,23.465829,32.985907,27.584803,0.402677,68.814960,1
997,3,98.197505,61.848646,21.875743,57.949824,21.456311,0.670626,62.615457,1
998,4,128.256681,83.592925,22.371875,42.656943,29.903630,0.607898,34.721735,0


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
X = df.drop(columns=['diagnosis'])
y = df['diagnosis']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
pipelines = {
    'Random Forest': Pipeline([('scaler', StandardScaler()), ('rf', RandomForestClassifier(random_state=42))]),
    'Logistic Regression': Pipeline([('scaler', StandardScaler()), ('lr', LogisticRegression(random_state=42))]),
    'XGBoost': Pipeline([('scaler', StandardScaler()), ('xgb', XGBClassifier(random_state=42))]),
    'Gradient Boosting': Pipeline([('scaler', StandardScaler()), ('gb', GradientBoostingClassifier(random_state=42))]),
    'Support Vector Machine': Pipeline([('scaler', StandardScaler()), ('svc', SVC(random_state=42))])
}

# Step 3: Train models and store accuracies
accuracies = {}
for model_name, pipeline in pipelines.items():
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies[model_name] = accuracy

# Step 4: Select the best model
best_model = max(accuracies, key=accuracies.get)
best_accuracy = accuracies[best_model]

In [20]:
print(f"The best model is {best_model} with accuracy {best_accuracy:.2f}")

The best model is Logistic Regression with accuracy 0.66


In [21]:
accuracies

{'Random Forest': 0.655,
 'Logistic Regression': 0.66,
 'XGBoost': 0.585,
 'Gradient Boosting': 0.635,
 'Support Vector Machine': 0.66}